In [3]:
import torch
torch.cuda.empty_cache()


In [1]:
!nvidia-smi

Sat Jul 20 15:21:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.183.01             Driver Version: 535.183.01   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650        Off | 00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8               3W /  50W |      8MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = AutoModelForCausalLM.from_pretrained("Majipa/text-to-SQL",
                                             device_map="cuda",
                                             torch_dtype="auto",
                                             quantization_config=quantization_config)

/home/sukuna/miniconda3/envs/LLM/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/sukuna/miniconda3/envs/LLM/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/home/sukuna/miniconda3/envs/LLM/lib/python3.11/site-packages/transformers/quantizers/auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attrib

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline 

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct") 

messages = [ 
    {"role": "system", "content": "You are a helpful text-to-SQL assistant."}, 
    {"role": "user", "content": "question: How many heads of the departments are older than 56 ? context: CREATE TABLE head (age INTEGER)"}, 
] 

pipe = pipeline( 
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
) 

generation_args = { 
    "max_new_tokens": 500, 
    "temperature": 0.7, 
    "return_full_text": False,
} 

output = pipe(messages, **generation_args) 
print(output[0]['generated_text'])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/sukuna/miniconda3/envs/LLM/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 SELECT COUNT(head) FROM head WHERE age > 56


In [3]:
from langchain.llms import HuggingFacePipeline
import torch 
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [7]:
pipe = pipeline( 
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_length=500,
    return_full_text=False,
) 

In [8]:
local_llm = HuggingFacePipeline(pipeline=pipe)

In [9]:
print(local_llm("question: select people whose name starts with s context: CREATE TABLE student(name VARCHAR(40))"))

 SELECT name FROM student WHERE name LIKE "s%"
